## Recomendación de peliculas

En este notebook exploramos un sistema de recomendación simple con la librerias TensorFlow + Keras. Los sistemas de recomendación se suelen componer de dos etapas:

La primera etapa se envarga de seleccionar un conjunto inicial de cientos de candidatos de entre todos los posibles. El principal objetivo de este modelo es filtrar eficientemente todos los candidatos en los que el usuario no está interesado.
En este notebooks, nos vamos a centrar en la primera etapa. La segunda etapa, de clasificación, toma las salidas de la primera etapa y las ajusta para seleccionar las mejores recomendaciones posibles

Vamos a construir y entrenar un modelo Two-Tower utilizando el conjunto de datos Movielens.

Vamos a:

    - Obtener nuestros datos y dividirlos en un conjunto de entrenamiento y uno de prueba.
    - Implementar un modelo de recuperación.
    - Ajustarlo y evaluarlo.

## Los datos

El conjunto de datos Movielens es un conjunto de datos del Grupo de investigación GroupLens. Contiene un conjunto de calificaciones dadas a películas por un conjunto de usuarios.

En este notebook, nos estamos centrando en un sistema de recuperación: un modelo que predice un conjunto de películas del catálogo que es probable que el usuario vea. A menudo, los datos implícitos son más útiles aquí, por lo que vamos a tratar Movielens como un sistema implícito. Esto significa que cada película que un usuario ha visto es un ejemplo positivo, y cada película que no ha visto es un ejemplo negativo implícito.

## Imports

In [72]:
import os
import pprint
import tempfile
import datetime
from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [73]:
import tensorflow_recommenders as tfrs

## Preparing the dataset

Utilizamos el conjunto de datos MovieLens de Tensorflow Datasets. movielens/100k_ratings contiene los datos de los ratings, y movielens/100k_movies contiene únicamente los datos solo de las películas.

In [74]:
ratings = tfds.load("movielens/100k-ratings", split="train")
movies = tfds.load("movielens/100k-movies", split="train")

El conjunto de datos de ratings devuelve un diccionario que incluye el ID de la película, el ID del usuario, la calificación asignada, la marca de tiempo, la información de la película y la información del usuario:

In [75]:
for x in ratings.take(1).as_numpy_iterator():
  pprint.pprint(x)

{'bucketized_user_age': 45.0,
 'movie_genres': array([7], dtype=int64),
 'movie_id': b'357',
 'movie_title': b"One Flew Over the Cuckoo's Nest (1975)",
 'raw_user_age': 46.0,
 'timestamp': 879024327,
 'user_gender': True,
 'user_id': b'138',
 'user_occupation_label': 4,
 'user_occupation_text': b'doctor',
 'user_rating': 4.0,
 'user_zip_code': b'53211'}


El conjunto de datos de películas contiene el ID de la película, el título de la película y datos sobre los géneros a los que pertenece.

In [76]:
for x in movies.take(1).as_numpy_iterator():
  pprint.pprint(x)

{'movie_genres': array([4], dtype=int64),
 'movie_id': b'1681',
 'movie_title': b'You So Crazy (1994)'}


Nos quedamos solo con los campos user_id y movie_title.

In [77]:
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
})
movies = movies.map(lambda x: x["movie_title"])

Para ajustar y evaluar el modelo, necesitamos dividir el conjunto de datos en un conjunto de entrenamiento y otro de evaluación. En un sistema de recomendación industrial, esto probablemente se haría en función del tiempo: los datos hasta el momento $T$ se utilizarían para predecir interacciones después de $T$.

En este ejemplo sencillo, utilizamos una particion aleatoria, asignando el 80% de las calificaciones en el conjunto de entrenamiento y el 20% en el conjunto de prueba.

In [78]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

También obtenemos los IDs de usuarios únicos y los títulos de películas presentes en los datos.

Esto es importante porque necesitamos poder mapear los valores brutos de nuestras características a vectores de embedding en nuestros modelos. Para hacer eso, necesitamos un vocabulario que asigne un valor bruto de una característica a un entero en un rango contiguo: esto nos permite buscar los embeddings correspondientes en nuestras tabla de embeddings.

In [79]:
movie_titles = movies.batch(1_000)
user_ids = ratings.batch(1_000_000).map(lambda x: x["user_id"])

unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

unique_movie_titles[:10]

array([b"'Til There Was You (1997)", b'1-900 (1994)',
       b'101 Dalmatians (1996)', b'12 Angry Men (1957)', b'187 (1997)',
       b'2 Days in the Valley (1996)',
       b'20,000 Leagues Under the Sea (1954)',
       b'2001: A Space Odyssey (1968)',
       b'3 Ninjas: High Noon At Mega Mountain (1998)',
       b'39 Steps, The (1935)'], dtype=object)

Implementación de un modelo

Elegir la arquitectura de nuestro modelo es una parte clave del modelado.

Dado que estamos construyendo un modelo Two-Tower, podemos construir cada torre por separado y luego combinarlas en el modelo final.

### La torre de consultas

In [80]:
embedding_dimension = 32

Valores más altos de dimensionalidad corresponderán a modelos que pueden ser más precisos, pero que también serán más lentos de ajustar y más propensos al overfitting.

El segundo paso es definir el modelo en sí. Aquí, vamos a utilizar las capas de preprocesamiento de Keras para convertir primero los IDs de usuario en enteros y luego convertirlos en embeddings de usuario a través de una capa de embedding.

In [81]:
user_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_user_ids, mask_token=None),
  # Añadimos un embedding adicional para tener en cuenta los tokens desconocidos.
  tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
])

Un modelo simple como este corresponde exactamente a un enfoque clásico de [matrix factorization](https://ieeexplore.ieee.org/abstract/document/4781121).

### La torre de candidatos

Hacemos el mismo proceso anterior con la torre de candidatos

In [70]:
movie_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_movie_titles, mask_token=None),
  tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
])

### Metricas

En nuestros datos de entrenamiento, tenemos pares positivos (usuario, película). Para determinar qué tan bueno es nuestro modelo, necesitamos comparar la puntuación de afinidad que el modelo calcula para este par con las puntuaciones de todos los demás candidatos posibles: si la puntuación para el par positivo es más alta que para todos los otros candidatos, nuestro modelo es preciso.

Para hacer esto, podemos usar la métrica tfrs.metrics.FactorizedTopK. Esta métrica tiene un unico parametro: el conjunto de datos de candidatos que se usan como negativos implícitos para la evaluación.

En nuestro caso, ese es el conjunto de datos movies, convertido en embeddings a través de nuestro modelo de películas:

In [84]:
metrics = tfrs.metrics.FactorizedTopK(
  candidates=movies.batch(128).map(movie_model)
)

### Función de perdida

El siguiente componente es la función de pérdida utilizada para entrenar nuestro modelo. TFRS tiene varias capas de pérdida y tareas para facilitar este proceso.

En este caso, utilizaremos el objeto Retrieval task: un wrapper conveniente que agrupa la función de pérdida y el cálculo de métricas:

In [85]:
task = tfrs.tasks.Retrieval(
  metrics=metrics
)

The task itself is a Keras layer that takes the query and candidate embeddings as arguments, and returns the computed loss: we'll use that to implement the model's training loop.

### El modelo

Ahora juntamos todo en un modelo. TFRS expone una clase base de modelo (tfrs.models.Model) que facilita la construcción de modelos: todo lo que necesitamos hacer es configurar los componentes en el método __init__ e implementar el método compute_loss, tomando las características en bruto y devolviendo un valor de pérdida.

El modelo base se encargará de crear el bucle de entrenamiento adecuado para ajustar nuestro modelo.

In [86]:
class MovielensModel(tfrs.Model):

  def __init__(self, user_model, movie_model):
    super().__init__()
    self.movie_model: tf.keras.Model = movie_model
    self.user_model: tf.keras.Model = user_model
    self.task: tf.keras.layers.Layer = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # Seleccionamos las características del usuario y las pasamos al modelo de usuario.
    user_embeddings = self.user_model(features["user_id"])
    # Seleccionamos las características de las películas y las pasamos al modelo de películas.
    positive_movie_embeddings = self.movie_model(features["movie_title"])

    return self.task(user_embeddings, positive_movie_embeddings)

## Ajuste y evaluación

Después de definir el modelo, podemos utilizar las rutinas estándar de ajuste y evaluación de Keras para entrenar y evaluar el modelo.

Primero, vamos a instanciar el modelo

In [102]:
model = MovielensModel(user_model, movie_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.5))

Luego, mezclamos, agrupamos en batches y almacenamos en caché los datos de entrenamiento y evaluación.

In [104]:
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

Luego entrenamos el modelo

In [105]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(cached_train, epochs=10, callbacks=[tensorboard_callback])

Epoch 1/10
10/10 [==============================] - 6s 440ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0015 - factorized_top_k/top_5_categorical_accuracy: 0.0117 - factorized_top_k/top_10_categorical_accuracy: 0.0249 - factorized_top_k/top_50_categorical_accuracy: 0.1231 - factorized_top_k/top_100_categorical_accuracy: 0.2265 - loss: 71531.3864 - regularization_loss: 0.0000e+00 - total_loss: 71531.3864
Epoch 2/10
10/10 [==============================] - 4s 449ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0021 - factorized_top_k/top_5_categorical_accuracy: 0.0160 - factorized_top_k/top_10_categorical_accuracy: 0.0350 - factorized_top_k/top_50_categorical_accuracy: 0.1752 - factorized_top_k/top_100_categorical_accuracy: 0.3069 - loss: 65564.8125 - regularization_loss: 0.0000e+00 - total_loss: 65564.8125
Epoch 3/10
10/10 [==============================] - 5s 457ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0026 - factorized_top_k/top_5_categorical_accurac

A medida que el modelo entrena, la pérdida disminuye y se actualiza un conjunto de métricas de recuperación top-k. Estas nos indican si el verdadero positivo se encuentra entre los elementos recuperados en el top-k del conjunto completo de candidatos. Por ejemplo, una métrica de precisión categórica top-5 de 0.2 nos indicaría que, en promedio, el verdadero positivo se encuentra entre los 5 elementos principales recuperados el 20% del tiempo.

Ahora, evaluamos el model en el conjunto de prueba:

In [106]:
model.evaluate(cached_test, return_dict=True, callbacks=[tensorboard_callback])

5/5 [==============================] - 1s 193ms/step - factorized_top_k/top_1_categorical_accuracy: 5.0000e-05 - factorized_top_k/top_5_categorical_accuracy: 9.0000e-04 - factorized_top_k/top_10_categorical_accuracy: 0.0036 - factorized_top_k/top_50_categorical_accuracy: 0.0743 - factorized_top_k/top_100_categorical_accuracy: 0.1877 - loss: 32696.2044 - regularization_loss: 0.0000e+00 - total_loss: 32696.2044


{'factorized_top_k/top_1_categorical_accuracy': 4.999999873689376e-05,
 'factorized_top_k/top_5_categorical_accuracy': 0.0008999999845400453,
 'factorized_top_k/top_10_categorical_accuracy': 0.0035500000230968,
 'factorized_top_k/top_50_categorical_accuracy': 0.07434999942779541,
 'factorized_top_k/top_100_categorical_accuracy': 0.18774999678134918,
 'loss': 29655.32421875,
 'regularization_loss': 0,
 'total_loss': 29655.32421875}

El rendimiento en el conjunto de prueba es mucho peor que en el conjunto de entrenamiento. Esto se debe a dos factores:

    - Es probable que nuestro modelo funcione mejor con los datos que ha visto porque puede memorizarlos. Este fenómeno de overfitting es especialmente fuerte cuando los modelos tienen muchos parámetros. Puede mitigarse mediante la regularización del modelo y el uso de características de usuarios y películas que ayuden al modelo a generalizar mejor a datos no vistos.

    - El modelo está volviendo a recomendar algunas de las películas que los usuarios ya han visto. Estas visualizaciones positivas conocidas pueden desplazar a las películas de prueba de las principales recomendaciones de top K.

El segundo fenómeno puede abordarse excluyendo las películas ya vistas de las recomendaciones de prueba.

## Haciendo predicciones
Ahora que tenemos un modelo, podemos hacer predicciones. usamos la capa tfrs.layers.factorized_top_k.BruteForce para hacerlo.

In [93]:
# Creamos un modelo que tome las características de la consulta.
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
# Y recomienda películas de todo el conjunto de datos de películas.
index.index_from_dataset(
  tf.data.Dataset.zip((movies.batch(100), movies.batch(100).map(model.movie_model)))
)

_, titles = index(tf.constant(["21"]))
print(f"Recommendations for user 212: {titles[0, :3]}")

Recommendations for user 212: [b"Wes Craven's New Nightmare (1994)" b'Hellraiser: Bloodline (1996)'
 b"Stephen King's The Langoliers (1995)"]
